In [1]:
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModel, AutoConfig
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
from torcheval.metrics import MulticlassConfusionMatrix,MulticlassF1Score
from tqdm import tqdm


import torch.nn as nn
import pandas as pd
import torch
import ipdb
import ast

# EDA, Data Formating and Preprocessing

In [2]:
data = pd.read_csv("topical_chat.csv")

In [3]:
data.head()

,conversation_id,message,sentiment
0,1,Are you a fan of Google or Microsoft?,Curious to dive deeper
1,1,Both are excellent technology they are helpful...,Curious to dive deeper
2,1,"I'm not a huge fan of Google, but I use it a...",Curious to dive deeper
3,1,Google provides online related services and p...,Curious to dive deeper
4,1,"Yeah, their services are good. I'm just not a...",Curious to dive deeper


In [4]:
data['sentiment'].unique()

array(['Curious to dive deeper', 'Happy', 'Neutral', 'Surprised',
       'Disgusted', 'Sad', 'Fearful', 'Angry'], dtype=object)

Impute Null rows which have nulls

In [5]:
len(data)

188378

In [6]:
data = data.dropna()

In [7]:
len(data)

188373

  We will create 3 differect datasets for the three different task

### Sentiment Analysis

In [8]:
sentiment_data = data[["message","sentiment"]]

In [9]:
sentiment_data.to_csv("dataset/sentiment_data.csv",index=False)

### Summary Generation and QA Dataset

In [10]:
summ_dataset = []
conv_id = []
for idx,cov in data.groupby(['conversation_id']):
    summ_dataset.append("\n".join(cov['message']).strip("\n"))
    conv_id.append(idx)
    # print(cov)
    # out = ''
    # for x in cov['message']:
    #     if x in not None:
    #         out+=x+'\n'
    # summ_dataset.append(out.strip('\n'))    
    # try:
    #     summ_dataset.append("\n".join(cov['message']).strip("\n"))
    # except:
    #     print(f"ERROR: In conversatoin id {idx}")
    #     out = ''
    #     for x in cov['message']:
    #         out += "\n"+str(x)
    #     out.strip('\n')
    #     print(out)
print(len(summ_dataset))

8628


In [11]:
pd.DataFrame({'conversation':summ_dataset[:100] , "conv_id":conv_id[:100]}).to_csv('dataset/100_conversation.csv',index=False)

In [12]:
pd.DataFrame({'conversation':summ_dataset[:100] , "conv_id":conv_id[:100]}).to_csv('dataset/conversation.csv',index=False)

In [15]:
# messages = [ msg.strip(' ') for msg in data['message'] ]
# qa_pair = []
# idx = 0
# while(idx<len(messages)):
#     pair = []
#     if messages[idx][-1]=='?':
#         pair.append(messages[idx])
#         pair.append(messages[idx+1])
#         qa_pair.append(pair)
#         idx+=1
#     idx+=1  

In [16]:
# len(qa_pair)

34303

In [27]:
# ques , ans = [],[]
# for q,a in qa_pair:
#     ques.append(q)
#     ans.append(a)

In [28]:
# pd.DataFrame({'Questions':ques,'Answers':ans}).to_csv('dataset/qa_dataset.csv',index=False)

## Model Sentiment Analysis

In [2]:
sentiment_data = pd.read_csv("dataset/sentiment_data.csv")

In [3]:
sentiment_data['sentiment'].unique()

array(['Curious to dive deeper', 'Happy', 'Neutral', 'Surprised',
       'Disgusted', 'Sad', 'Fearful', 'Angry'], dtype=object)

In [4]:
str2cls = {
 'Curious to dive deeper':0, 
 'Happy':1, 
 'Neutral':2, 
 'Surprised':3,
 'Disgusted':4, 
 'Sad':5,
 'Fearful':6,
 'Angry':7
}

In [5]:
sentiment_data['sentiment'] = sentiment_data['sentiment'].replace(str2cls)

In [6]:
sentiment_data['sentiment'].unique()

array([0, 1, 2, 3, 4, 5, 6, 7])

In [7]:
sentiment_data['sentiment'].value_counts()

0    80887
2    41367
3    30637
1    29615
5     2533
4     1432
6     1026
7      876
Name: sentiment, dtype: int64

In [8]:
cls_dist = (sentiment_data['sentiment'].value_counts()/len(sentiment_data))

In [9]:
cls_dist

0    0.429398
2    0.219602
3    0.162640
1    0.157215
5    0.013447
4    0.007602
6    0.005447
7    0.004650
Name: sentiment, dtype: float64

The classes are higly imbalanced so some oversampling might be needed to make the data balanced

In [10]:
cls_weights = cls_dist**(-1)/100

In [11]:
cls_weights * sentiment_data['sentiment'].value_counts()

0    1883.73
2    1883.73
3    1883.73
1    1883.73
5    1883.73
4    1883.73
6    1883.73
7    1883.73
Name: sentiment, dtype: float64

In [12]:
cls_weights = { key:val for key,val in zip(cls_weights.index,cls_weights) }

In [13]:
cls_weights

{0: 0.023288414701991663,
 2: 0.045537022264123574,
 3: 0.06148545875901688,
 1: 0.0636072936012156,
 5: 0.7436754836162653,
 4: 1.315453910614525,
 6: 1.8359941520467837,
 7: 2.150376712328767}

In [14]:
class TaskADataset(Dataset):
    
    def over_sampling(self,data_base,weights):
        # Oversampling is done so that the dataset is balanced
        data = []
        for idx,gp in data_base.groupby(['sentiment']):
            out = gp.sample(frac=weights[idx], replace=True,random_state=43)
            data.append(out)
        data = pd.concat(data)
        print(data.describe())
        return data

    def __init__(self, data, tokenizer, max_len:int, weights={i:1 for i in range(8)}):
        self.data = self.over_sampling(data,weights)
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        data = self.data.iloc[index]
        text = data.message
        label = data.sentiment
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [15]:
class TaskA_Model(nn.Module):

    def __init__(self, model_name, num_classes):
        super(TaskA_Model, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.L1 = nn.Linear(768, 200)
        self.L2 = nn.Linear(200,25)
        self.classifier = nn.Linear(25,num_classes)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, input_ids, attention_mask):

        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        outputs = self.L1(outputs.pooler_output)
        outputs = self.L2(outputs)
        class_weights = self.classifier(outputs)
        probs = self.softmax(class_weights)
        return probs

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Need to look at this code and modify this 

In [36]:
def val_model(model,criterion,metrics,th,val_dataloader,min_val_loss=-1):
    
    
    model.eval()
    predictions = []
    total_val_loss = 0
    for batch_idx,data in tqdm(enumerate(val_dataloader),total = len(val_dataloader)):
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        label = data['label'].to(device)
        output = model(input_ids, attention_mask)
        pred = torch.zeros(len(label))
        pred[output[:,1]>th] = 1
        loss = criterion(output,label)
        pred = pred.to(torch.int64)
        label = label.to(torch.int64)
        label = label.to('cpu')
        predictions.append(pred)
        
        for m in metrics:
            m.update(pred,label)
        
        total_val_loss+=loss.item()
    val_loss = total_val_loss/len(val_dataloader)
    if val_loss<min_val_loss:
        min_val_loss = val_loss
        torch.save(model.state_dict(),"./model_bert_freeze")
        print("Model Saved")

    print("Macro F1: ",metrics[0].compute())
    print("Micro F1: ",metrics[1].compute())
    print("Confusion Matric: \n",metrics[2].compute())
    if len(metrics)>3:
        print("Accuracy: ",metrics[3].compute())
    for m in metrics:
            m.reset()
    return val_loss


def train_model(model,optimizer,criterion,scheduler,num_epoch,F1_macro_metric,F1_micro_metric,confusion_matrix,th,val_dataloader):
    min_val_loss = 1000
    for epoch in range(num_epoch):
        model.train()
        tot_loss = 0
        for batch_idx, data in tqdm(enumerate(train_loader),total=len(train_loader)):
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            label = data['label'].to(device)
            optimizer.zero_grad()
            output = model(input_ids, attention_mask)
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            tot_loss += loss.item()
        
        val_loss = val_model( model,
                              criterion,
                              F1_macro_metric,
                              F1_micro_metric,
                              confusion_matrix,
                              th,
                              val_dataloader,
                              min_val_loss)
        
        min_val_loss = min(min_val_loss,val_loss)
        print(f"Epoch Loss : {tot_loss/len(train_loader)} Val Loss: {val_loss}")
        scheduler.step(tot_loss/len(train_loader))


## Split the data into train val and test

In [17]:
from sklearn.model_selection import train_test_split

train,val = train_test_split(sentiment_data,test_size=0.1,stratify=sentiment_data['sentiment'],random_state=43)

In [18]:
train['sentiment'].value_counts()/len(train)

0    0.429398
2    0.219601
3    0.162639
1    0.157212
5    0.013449
4    0.007603
6    0.005444
7    0.004654
Name: sentiment, dtype: float64

In [19]:
val['sentiment'].value_counts()/len(val)

0    0.429398
2    0.219609
3    0.162650
1    0.157235
5    0.013430
4    0.007591
6    0.005468
7    0.004618
Name: sentiment, dtype: float64

## Setup the model and init weights

In [20]:
import os

In [21]:
max_len = max( [ len(i.split()) for i in train["message"] ] )+100
print("Max length of message: ", max_len)

model_name = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
nclasses = len(cls_weights)
model = TaskA_Model(model_name,num_classes=nclasses).to(device)

Max length of message:  227


In [23]:
if os.path.exists('model_bert_freeze'):
    model.load_state_dict(torch.load('model_bert_freeze'))
    print("Pretrained model exists and loaded")

Pretrained model exists and loaded


## If only the linear layer traning is the target

In [23]:
# for params in model.bert.parameters():
#     params.requires_grad = False

## Init the params for bert model

In [26]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',threshold=1e-3)
num_epoch = 10
th = 0.5
F1_macro_metric = MulticlassF1Score(average="macro", num_classes=nclasses)
F1_micro_metric = MulticlassF1Score(average="micro", num_classes=nclasses)
confusion_matrix = MulticlassConfusionMatrix(num_classes=nclasses)

train_dataset = TaskADataset(train, tokenizer, max_len,weights=cls_weights)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

val_dataset = TaskADataset(val,tokenizer,max_len)
val_dataloader = DataLoader(val_dataset,batch_size=16)


train_model(model = model,
            optimizer=optimizer,
            criterion=criterion,
            scheduler=scheduler,
            num_epoch=num_epoch,
            F1_macro_metric=F1_macro_metric,
            F1_micro_metric=F1_micro_metric,
            confusion_matrix=confusion_matrix,
            th=th,
            val_dataloader=val_dataloader)

          sentiment
count  13564.000000
mean       3.500664
std        2.291469
min        0.000000
25%        2.000000
50%        4.000000
75%        6.000000
max        7.000000
          sentiment
count  18838.000000
mean       1.247054
std        1.350861
min        0.000000
25%        0.000000
50%        1.000000
75%        2.000000
max        7.000000


100%|████████████████████████| 1178/1178 [03:01<00:00,  6.49it/s]


Model Saved
Macro F1:  tensor(0.1210)
Micro F1:  tensor(0.4477)
Confusion Matric: 
 tensor([[7.4670e+03, 6.2200e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.9950e+03, 9.6700e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [3.5570e+03, 5.8000e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [2.8190e+03, 2.4500e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.4200e+02, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [2.4400e+02, 9.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.0000e+02, 3.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [8.6000e+01, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+0

100%|████████████████████████| 1178/1178 [03:02<00:00,  6.47it/s]


Model Saved
Macro F1:  tensor(0.1236)
Micro F1:  tensor(0.4385)
Confusion Matric: 
 tensor([[6.9060e+03, 1.1830e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.6070e+03, 1.3550e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [3.1230e+03, 1.0140e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [2.5580e+03, 5.0600e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.3500e+02, 8.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [2.3200e+02, 2.1000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [9.7000e+01, 6.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [8.4000e+01, 3.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+0

...

Macro F1:  tensor(0.1220)
Micro F1:  tensor(0.4355)
Confusion Matric: 
 tensor([[6.9630e+03, 1.1260e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.7210e+03, 1.2410e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [3.3760e+03, 7.6100e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [2.6380e+03, 4.2600e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.4000e+02, 3.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [2.4500e+02, 8.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [9.8000e+01, 5.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [7.9000e+01, 8.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00]])
Epoch: 

100%|████████████████████████| 1178/1178 [03:01<00:00,  6.48it/s]


Macro F1:  tensor(0.1227)
Micro F1:  tensor(0.4434)
Confusion Matric: 
 tensor([[7.2210e+03, 8.6800e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.8310e+03, 1.1310e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [3.4980e+03, 6.3900e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [2.6770e+03, 3.8700e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.3800e+02, 5.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [2.4400e+02, 9.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [9.9000e+01, 4.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [8.6000e+01, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00]])
Epoch: 

100%|████████████████████████| 1178/1178 [03:01<00:00,  6.50it/s]

Macro F1:  tensor(0.1206)
Micro F1:  tensor(0.4364)
Confusion Matric: 
 tensor([[7.0870e+03, 1.0020e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.8280e+03, 1.1340e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [3.4480e+03, 6.8900e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [2.6920e+03, 3.7200e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.4100e+02, 2.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [2.4900e+02, 4.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.0000e+02, 3.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [8.3000e+01, 4.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00]])
Epoch: 

In [ ]:
# Change bert to S-sbert roberta large or some bigger mode
# Llama 2 find tune for classification

# F1 Macro , F1-Micro,  Confusion_matrix, weigted F1


In [ ]:
# Future work
1. Classificatioasn io

## Model Summary Generation

In [2]:
## Mistral 7B with prompt tuning with conversation 
## Use sometthing like bert score between the conversation and the summary 
## Some modified form of blue score or other can be used (where we remove the stopword )

In [3]:
import pandas as pd
conv_data = pd.read_csv('dataset/100_conversation.csv')

In [4]:
conv_data.head()

,conversation,conv_id
0,Are you a fan of Google or Microsoft?\nBoth ar...,1
1,do you like dance?\nYes I do. Did you know Br...,2
2,Hey what's up do use Google very often?I reall...,3
3,Hi! do you like to dance?\nI love to dance a ...,4
4,do you like dance?\nI love it. Did you know Br...,5


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto
model_name = "meta-llama/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.to(device)

In [ ]:
prompt = "My favourite condiment is"

model_inputs = tokenizer([prompt], return_tensors="pt").to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

## Model QA 

In [61]:
## GPT-2 Base / Llama 2 Lora fine tune on the qa pairs
## Use something like rouge , Bleu to evaluete the models 